In [2]:
import numpy as np
import torch
%matplotlib inline
from matplotlib import pyplot as plt
import tqdm
import torch.nn.functional as F
import torch.nn as nn
import pickle
import fb_utils as fb
from torch.utils.data import DataLoader
import random
import pandas
import os

from model import TDFilterbank
from dataset import TinySol
from losses import KappaLoss

In [ ]:
# set seed
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(0)
np.random.seed(0)

In [12]:

tiny_sol_csv = '/Users/felixperfler/Documents/ISF/Random Filterbanks/TinySOL_metadata.csv'
TinySol_df = pandas.read_csv(tiny_sol_csv)

instrument_classes = TinySol_df['Instrument (abbr.)'].unique()
# map instrument classes to integers
instrument_to_int = {instrument: i for i, instrument in enumerate(instrument_classes)}
int_to_instrument = {i: instrument for i, instrument in enumerate(instrument_classes)}


In [ ]:
config = {
        "N": 2**12,
        "J": 96,
        "T": 1024,
        "sr": 16000,
        "fmin": 64,
        "fmax": 8000,
        "stride": 512,
        "batch_size": 64,
        "epochs": 100,
    }

# info_csv_path="/Users/felixperfler/Documents/ISF/Random-Filterbanks/TinySOL_metadata.csv"
# data_dir="/Users/felixperfler/Documents/ISF/Random-Filterbanks/TinySOL2020"
info_csv_path="/Users/Dane/Documents/GitHub/Random-Filterbanks/TinySOL_metadata.csv"
data_dir="/Users/Dane/Documents/GitHub/Random-Filterbanks/TinySOL2020"


random_filterbank = fb.random_filterbank(config["N"], config["J"], config["T"], tight=False, support_only=True)
target = 'VQT'
# get current working directory of file
cwd = os.path.abspath('')

with open(cwd+'/targets/'+target+'.pkl', 'rb') as fp:
    target_filterbank = pickle.load(fp)["freqz"]
    target_filterbank = torch.from_numpy(target_filterbank.T)

# device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'